In [0]:
#install.packages('PerformanceAnalytics')
#install.packages('caret')
#install.packages('e1071')
#install.packages('mlbench')
#install.packages('randomForest')
library(PerformanceAnalytics)
library(caret)
library(randomForest)
library(e1071)
library(pROC)

# normalize features to (0:1)
normalize <- function (x) {
  return ((x - min(x)) / (max(x) - min(x)))
}

data = read.csv(file="../input/forest-cover-type-prediction/train.csv", header=TRUE, sep=',')

dim(data)
str(data)
summary(data)
table(is.na(data))
klasy <- aggregate(data$Cover_Type, by=list(data$Cover_Type), FUN=length)

# drop unnecesarry columns
data <- subset(data, select = -c(Id, Soil_Type7, Soil_Type15))
data$Cover_Type <- as.factor(data$Cover_Type)

# check and delete highly correlated features
correlationMatrix <- cor(data[,1:10])
print(correlationMatrix)
highlyCorrelated <- findCorrelation(correlationMatrix, cutoff=0.75, verbose=TRUE)
print(highlyCorrelated)

data <- subset(data, select = -c(Hillshade_3pm))

# normalize columns 1:10
i=1
for (i in 1:10) {
  data[,i] <- normalize(data[,i])
}


# split to train and test
smp_size <- floor(0.8 * nrow(data))
set.seed(123)
train_ind <- sample(seq_len(nrow(data)), size = smp_size)

data_train <- data[train_ind, ]
data_test <- data[-train_ind, ]

######################################## RANDOM FOREST

# finding best 'mtry' parameter
# a=c()
# for (i in 1:25) {
#  rfmodel <- randomForest(Cover_Type ~., data=data_train, ntree=500, importance=TRUE, mtry=i)
#  predValid <- predict(rfmodel, data_test, type="class")
#  a[i] <- mean(predValid == data_test$Cover_Type)
#  message("i = ", i, ", a = ", a[i])
# }
# a
# plot(1:25, a, xlab="Liczba losowanych atrybutĂłw", ylab="SkutecznoĹ›Ä‡ predykcji", main="SkutecznoĹ›Ä‡ modelu w zaleznoĹ›ci od liczby losowanych atrybutĂłw")


# RF model
random_forest <- randomForest(Cover_Type ~., data=data_train, ntree=500, importance=TRUE, mtry=20)

# check on test dataset
rf_prediction <- predict(random_forest, data_test, type="class")
confusionMatrix(rf_prediction, data_test$Cover_Type)
# feature importance
importance_1 <- importance(random_forest, type=1)
importance_2 <- importance(random_forest, type=2)
varImpPlot(random_forest, n.var=20, type=1)
varImpPlot(random_forest, n.var=20, type=2)

rf_roc <- multiclass.roc(data_test$Cover_Type, factor(rf_prediction, ordered=TRUE))
print(rf_roc)


# ROC for every class (one vs others)
rf_prediction_prob <- as.data.frame(predict(random_forest, data_test, type="prob"))
rf_prediction_prob$predict <-names(rf_prediction_prob)[1:7][apply(rf_prediction_prob[,1:7], 1, which.max)]
rf_prediction_prob$observed <- data_test$Cover_Type
colnames(rf_prediction_prob)[which(names(rf_prediction_prob)==1)]<-"a"
colnames(rf_prediction_prob)[which(names(rf_prediction_prob)==2)]<-"b"
colnames(rf_prediction_prob)[which(names(rf_prediction_prob)==3)]<-"c"
colnames(rf_prediction_prob)[which(names(rf_prediction_prob)==4)]<-"d"
colnames(rf_prediction_prob)[which(names(rf_prediction_prob)==5)]<-"e"
colnames(rf_prediction_prob)[which(names(rf_prediction_prob)==6)]<-"f"
colnames(rf_prediction_prob)[which(names(rf_prediction_prob)==7)]<-"g"
roc1 <- roc(ifelse(rf_prediction_prob$observed==1, "1", "rest"), as.numeric(rf_prediction_prob$a))
roc2 <- roc(ifelse(rf_prediction_prob$observed==2, "2", "rest"), as.numeric(rf_prediction_prob$b))
roc3 <- roc(ifelse(rf_prediction_prob$observed==3, "3", "rest"), as.numeric(rf_prediction_prob$c))
roc4 <- roc(ifelse(rf_prediction_prob$observed==4, "4", "rest"), as.numeric(rf_prediction_prob$d))
roc5 <- roc(ifelse(rf_prediction_prob$observed==5, "5", "rest"), as.numeric(rf_prediction_prob$e))
roc6 <- roc(ifelse(rf_prediction_prob$observed==6, "6", "rest"), as.numeric(rf_prediction_prob$f))
roc7 <- roc(ifelse(rf_prediction_prob$observed==7, "7", "rest"), as.numeric(rf_prediction_prob$g))
plot(roc1, ylab="True Positive Rate", xlab="False Positive Rate", main="Krzywe ROC lasu losowego")
lines(roc2, col="blue")
lines(roc3, col="red")
lines(roc4, col="darkgreen")
lines(roc5, col="orange")
lines(roc6, col="darkmagenta")
lines(roc7, col="cyan")
legend("bottomright", legend=c(1, 2, 3, 4, 5, 6, 7), col=c("black", "blue", "red", "darkgreen", "orange", "darkmagenta", "cyan"), lty=1)

######################################## SVM
# drop 1 hot encoded features
dane_bez_1hot <- data[-c(10:51)]
# split new dataset
dane_bez_1hot_train <- dane_bez_1hot[train_ind, ]
dane_bez_1hot_test <- dane_bez_1hot[-train_ind, ]

# tune SVM
#tuned_svm_parameters <- tune.svm(Cover_Type ~., data=dane_bez_1hot_train, gamma=10^(-5:-1), cost=10^(-3:1))

# save best model to a file
#saveRDS(tuned_svm_parameters$best.model, './best_svm_model.rds')

# read model from a file
best_svm_model <- readRDS('../input/forest-cover-type-prediction/best_svm_model.rds')
# same as in RF
svm_prediction <- predict(best_svm_model, data_test, type="class")
confusionMatrix(svm_prediction, data_test$Cover_Type)
 
svm_roc <- multiclass.roc(data_test$Cover_Type, factor(svm_prediction, ordered=TRUE))
print(svm_roc)


######################################## NAIVE BAYES
# same as before
naive_bayes <- naiveBayes(Cover_Type ~., data=dane_bez_1hot_train)
nb_prediction <- predict(naive_bayes, dane_bez_1hot_test, type="class")
confusionMatrix(nb_prediction, dane_bez_1hot_test$Cover_Type)
nb_roc <- multiclass.roc(data_test$Cover_Type, factor(nb_prediction, ordered=TRUE))
print(nb_roc)


test = read.csv(file="../input/forest-cover-type-prediction/test.csv", header=TRUE, sep=',')
test$Cover_Type<-predict(random_forest,test)
output<-data.frame(test$Id,test$Cover_Type)
colnames(output) <- c("Id", "Cover_Type")
write.csv(output,file="submission.csv",row.names=FALSE)